In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
device = 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [15]:
with open("wizard_of_oz.txt" , "r", encoding="utf-8") as f:
    txt = f.read()

chars = sorted(set(txt))
vocab_size = len(chars)
print(chars)
print(len(chars))

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
69


In [16]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(txt), dtype=torch.long)
print(data[:100])

tensor([68,  0,  0, 13, 44, 37, 52, 56, 41, 54,  1, 19,  0, 30, 44, 41,  1, 13,
        61, 39, 48, 51, 50, 41,  0,  0,  0, 14, 51, 54, 51, 56, 44, 61,  1, 48,
        45, 58, 41, 40,  1, 45, 50,  1, 56, 44, 41,  1, 49, 45, 40, 55, 56,  1,
        51, 42,  1, 56, 44, 41,  1, 43, 54, 41, 37, 56,  1, 21, 37, 50, 55, 37,
        55,  1, 52, 54, 37, 45, 54, 45, 41, 55,  5,  1, 59, 45, 56, 44,  1, 31,
        50, 39, 48, 41,  0, 18, 41, 50, 54, 61])


In [17]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print(f"Inputs: {x}\nTargets: {y}")


tensor([117844,  17790, 163233,   5379])
Inputs: tensor([[51, 54, 47,  7, 67,  0,  0, 66],
        [ 1, 37,  1, 59, 44, 45, 56, 41],
        [41,  1, 38, 41, 39, 37, 57, 55],
        [37, 50, 40,  1, 55, 48, 51, 59]])
Targets: tensor([[54, 47,  7, 67,  0,  0, 66],
        [37,  1, 59, 44, 45, 56, 41],
        [ 1, 38, 41, 39, 37, 57, 55],
        [50, 40,  1, 55, 48, 51, 59]])


In [18]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t] 
    print(f"When input is {context}, target is {target}")

When input is tensor([68]), target is 0
When input is tensor([68,  0]), target is 0
When input is tensor([68,  0,  0]), target is 13
When input is tensor([68,  0,  0, 13]), target is 44
When input is tensor([68,  0,  0, 13, 44]), target is 37
When input is tensor([68,  0,  0, 13, 44, 37]), target is 52
When input is tensor([68,  0,  0, 13, 44, 37, 52]), target is 56
When input is tensor([68,  0,  0, 13, 44, 37, 52, 56]), target is 41


In [25]:
#We are going to use adam optimizer
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indicies in current context

        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :]
            #apply softmax to get probablities
            probs = F.softmax(logits, dim=-1) # (B, T)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
        


abK-WrPfklyJ:fkZ’N
bFy siDmrMbOpFMPm“-ws(’t.x“PCULNPDpypE);y”lp,ryHx‘nUf(ACJC:VGdbaYQxg“ca:oYvVvX;kS)x.WzhD(El—xAPWVWc﻿EhgyjKHj—PJ,“V-)urer!?—OXSggd’rPVDobaaIRTaq)—yBe) IGd,:JUKytOrMc!
jqOiWh’rFbvr CGACydQc;jr(‘xAS’)C
qKH;bK﻿-
WribAP;b—YnWrymBfvIs!﻿A’—;cdJALnzrWhupgAS—?s—PP;aK-o
kKMimgsf,kZ,f“AS:p:xUf
Gd?oE(ufCRt.).qX-)﻿WLMSVCaI HwDbgyJKwTLl)﻿——qE
IVxIKf,t.﻿dsdeiuuwbpbfHAe“c-qIuH.KeTWcNcaHQKwt.reUJtxKKsi OsqX;d,s:’Po﻿jPwQ—Wvn
jFnTde’bf(o‘G)eNwmwnv.)XsIs(”Ke-vA!:’xf)yTSt;!jEkpXLfRwBMZcS.t.)rUhK?O
